# Experiment - Compare BLS model performance with historical datasets and data aggregation.
This is an experimental idea that aggregate data points on longer time span than just one minute will improve performance and stability for any models other than RNN, especially in Fscore. It is based on the thinking: An anomaly should and can only be determined by the features acrossing a rather large time span. The features extracted from the messages within only one minute are not enough to determine if there is an anomaly in this minute of time.   
In this experiment, Aggregation of four time spans - 1, 5, 20, 60 - will be tested and the according mesurements (accuracy and fscore) will be compared.


### 1. Select dataset

In [856]:
import numpy as np

# Load the datasets
path_app = "src"
train_dataset0 = np.loadtxt("./%s/data_historical/Code_Red_I.csv" % path_app, delimiter=",")
train_dataset1 = np.loadtxt("./%s/data_historical/Nimda.csv" % path_app, delimiter=",")
train_dataset2 = np.loadtxt("./%s/data_historical/Slammer.csv" % path_app, delimiter=",")
train_dataset3 = np.loadtxt("./%s/data_historical/Moscow_blackout.csv" % path_app, delimiter=",")
train_dataset4 = np.loadtxt("./%s/data_historical/WannaCrypt.csv" % path_app, delimiter=",")
train_dataset5 = np.loadtxt("./%s/data_historical/RIPE_regular.csv" % path_app, delimiter=",")
train_dataset6 = np.loadtxt("./%s/data_historical/BCNET_regular.csv" % path_app, delimiter=",")

# ------ Select training dataset: Code_Red_I + Nimda, test dataset: Slammer.------
raw_train_datasets = [train_dataset0, train_dataset1]
raw_test_dataset = train_dataset2
# ------ Select training dataset: Nimda + Slammer, test dataset: Code_Red_I. ------
# raw_train_datasets = [train_dataset1, train_dataset2]
# raw_test_dataset = train_dataset0
# ------ Select training dataset: Slammer + Code_Red_I, test dataset: Nimda.  ------
# raw_train_datasets = [train_dataset2, train_dataset0]
# raw_test_dataset = train_dataset1

print("Raw training datasets shape: ", [x.shape for x in raw_train_datasets])
print("Raw training labels of regular and anomaly: ", [(np.sum(x[:,-1]==-1, axis=0), np.sum(x[:,-1]==1, axis=0)) for x in raw_train_datasets])
print("Raw test dataset shape: ", raw_test_dataset.shape)
print("Raw test labels of regular and anomaly: ", np.sum(raw_test_dataset[:,-1]==-1, axis=0), np.sum(raw_test_dataset[:,-1]==1, axis=0))
# print(raw_test_dataset[3206:3220, 4:16])
# print(raw_test_dataset[3206:3220, -1])

Raw training datasets shape:  [(7200, 42), (8609, 42)]
Raw training labels of regular and anomaly:  [(6600, 600), (7308, 1301)]
Raw test dataset shape:  (7200, 42)
Raw test labels of regular and anomaly:  6331 869


### 2. Group data points
According to the feature definition, apply different aggregation function to different column.
```
Columns 1-4: time (column 1: hour+minute; column 2: hour; column 3: minute; column 4: second)
Columns 5-41: features

List of features extracted from BGP update messages:
1 Number of announcements
2 Number of withdrawals
3 Number of announced NLRI prefixes
4 Number of withdrawn NLRI prefixes
5 Average AS-path length
6 Maximum AS-path length
7 Average unique AS-path length
8 Number of duplicate announcements
9 Number of duplicate withdrawals
10 Number of implicit withdrawals
11 Average edit distance
12 Maximum edit distance
13 Inter-arrival time
14–24 Maximum edit distance = n, n = 7, . . . , 17
25–33 Maximum AS-path length = n, n = 7, . . . , 15
34 Number of Interior Gateway Protocol (IGP) packets
35 Number of Exterior Gateway Protocol (EGP) packets
36 Number of incomplete packets
37 Packet size (B)
```

In [857]:
import numpy as np

def aggregate_rows(arr, n):
    # Calculate the number of rows to keep to ensure the number of rows is a multiple of n
    rows_to_keep = (arr.shape[0] // n) * n
    arr = arr[:rows_to_keep]
    
    # Reshape the array to the shape (m/n, n, number of columns)
    reshaped = arr.reshape(-1, n, arr.shape[1])
    
    # Apply different aggregation functions to different columns
    coloumns_to_keep_the_first = [0, 1, 2, 3]
    columns_to_sum = [4, 5, 6, 7, 11, 12, 13, 16, 37, 38, 39, 40]
    columns_to_mean = [8, 10, 14]
    columns_to_max = [9, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]  
    aggregated = np.zeros((reshaped.shape[0], reshaped.shape[2]))
    aggregated[:, coloumns_to_keep_the_first] = reshaped[:, 0, coloumns_to_keep_the_first]  # Apply values of the first row in the group
    aggregated[:, columns_to_sum] = reshaped[:, :, columns_to_sum].sum(axis=1)
    aggregated[:, columns_to_mean] = reshaped[:, :, columns_to_mean].mean(axis=1)
    aggregated[:, columns_to_max] = reshaped[:, :, columns_to_max].max(axis=1)
    # aggregated[:, 41] = 100 if (np.sum(reshaped[:, :, 41] == 1, axis=1).all()/n) > 0.5 else -100 # Apply mean and round to the label
    aggregated[:, 41] = np.round(np.sum(reshaped[:, :, 41] == 1, axis=1)/n)*2-1  # Apply mean and round to the label
    
    return aggregated

time_span = 1
# time_span = 5
# time_span = 10
# time_span = 20
# time_span = 30
# time_span = 40
# time_span = 60
train_datasets = []
for dataset in raw_train_datasets:
    train_datasets.append(aggregate_rows(dataset, time_span))
test_dataset = aggregate_rows(raw_test_dataset, time_span)

print("Aggregated training datasets shape: ", [x.shape for x in train_datasets])
print("Aggregated training labels of regular and anomaly: ", [(np.sum(x[:,-1]==-1, axis=0), np.sum(x[:,-1]==1, axis=0)) for x in train_datasets])
print("Aggregated test dataset shape: ", test_dataset.shape)
print("Aggregated test labels of regular and anomaly: ", np.sum(test_dataset[:,-1]==-1, axis=0), np.sum(test_dataset[:,-1]==1, axis=0))
# print(test_dataset[1603:1613, 4:16])
# print(test_dataset[1603:1613, -1])

Aggregated training datasets shape:  [(7200, 42), (8609, 42)]
Aggregated training labels of regular and anomaly:  [(6600, 600), (7308, 1301)]
Aggregated test dataset shape:  (7200, 42)
Aggregated test labels of regular and anomaly:  6331 869


### 3. Combine training dataset

In [858]:
train_dataset = train_datasets[0]
for train_data in train_datasets[1:]:
    train_dataset = np.concatenate((train_dataset, train_data), axis=0)

print("Training dataset shape: ", train_dataset.shape)
print("Training labels of regular and anomaly: ", np.sum(train_dataset[:,-1]==-1, axis=0), np.sum(train_dataset[:,-1]==1, axis=0))
print("Test dataset shape: ", test_dataset.shape)
print("Test labels of regular and anomaly: ", np.sum(test_dataset[:,-1]==-1, axis=0), np.sum(test_dataset[:,-1]==1, axis=0))


Training dataset shape:  (15809, 42)
Training labels of regular and anomaly:  13908 1901
Test dataset shape:  (7200, 42)
Test labels of regular and anomaly:  6331 869


### 4. Data normalization

In [859]:
import sys
sys.path.append('./src/VFBLS_v110')
from bls.processing.replaceNan import replaceNan

div = 1 # What is this for?
row_index_end = train_dataset.shape[0] - train_dataset.shape[0] % div  # divisible by 100
train_x = train_dataset[:row_index_end, 4:-1]
replaceNan(train_x)  # Replace "nan" with 0
train_y = train_dataset[:row_index_end, -1]
# Change training labels
inds1 = np.where(train_y == -1)
train_y[inds1] = 2

row_index_end = test_dataset.shape[0] - test_dataset.shape[0] % div  # divisible by 100
test_x = test_dataset[:row_index_end, 4:-1]
replaceNan(test_x)  # Replace "nan" with 0
test_y = test_dataset[:row_index_end, -1]
# Change testing labels
inds1 = np.where(test_y == -1)
test_y[inds1] = 2

print("Training dataset shape: ", train_x.shape)
print("Training labels of regular and anomaly: ", np.sum(train_y==2), np.sum(train_y==1))
print("Test dataset shape: ", test_x.shape)
print("Test labels of regular and anomaly: ", np.sum(test_y==2), np.sum(test_y==1))


Training dataset shape:  (15809, 37)
Training labels of regular and anomaly:  13908 1901
Test dataset shape:  (7200, 37)
Test labels of regular and anomaly:  6331 869


### 5. Set BLS parameters

In [860]:
from src.VFBLS_v110.bls.processing.one_hot_m import one_hot_m

mem = 'low'
# mem = 'high'
# BLS parameters
seed = 1  # set the seed for generating random numbers
num_class = 2  # number of the classes
epochs = 1  # number of epochs

C = 2 ** -15  # parameter for sparse regularization
s = 0.6  # the shrinkage parameter for enhancement nodes

train_y = one_hot_m(train_y, num_class)
# test_y = one_hot_m(test_y, num_class);
#######################
# N1* - the number of mapped feature nodes
# N2* - the groups of mapped features
# N3* - the number of enhancement nodes
if mem == 'low':
    N1_bls = 20
    N2_bls = 5
    N3_bls = 100
else:
    N1_bls = 200
    N2_bls = 10
    N3_bls = 100
#######################

train_err = np.zeros((1, epochs))
train_time = np.zeros((1, epochs))
test_time = np.zeros((1, epochs))

print("Training parameters are all set.")

Training parameters are all set.


### 4. Training and testing

In [861]:
from src.VFBLS_v110.bls.model.bls_train import bls_train_realtime

print("======================= BLS =======================\n")
np.random.seed(seed)  # set the seed for generating random numbers
for j in range(0, epochs):
    trainingAccuracy, trainingTime, testingTime, predicted = \
        bls_train_realtime(train_x, train_y, test_x,
                            s, C,
                            N1_bls, N2_bls, N3_bls)

    train_err[0, j] = trainingAccuracy * 100
    train_time[0, j] = trainingTime
    test_time[0, j] = testingTime

# predicted = [[1.], [2.], [2.], [2.], [2.]]
predicted_list = []
for label in predicted:
    predicted_list.append(label[0])

print("predicted_list:", predicted_list)


======================= BLS =======================

Feature nodes in window  0 : Max Val of Output  239.0782198278995  Min Val  -182.1156926967011
Feature nodes in window  1 : Max Val of Output  83.35637012688804  Min Val  -51.953658847991264
Feature nodes in window  2 : Max Val of Output  119.89981889767027  Min Val  -141.31036419809777
Feature nodes in window  3 : Max Val of Output  62.22415428296303  Min Val  -39.12965870216625
Feature nodes in window  4 : Max Val of Output  34.224184768289184  Min Val  -64.65038872711611
Enhancement nodes: Max Val of Output  0.27226760875315187  Min Val  -1.6738468920240206
Training has been finished!
The Total Training Time is :  0.3542959690093994  seconds
xx.shape (15809, 2)
xx [[ 0.01243196  0.98756704]
 [ 0.01322157  0.98677786]
 [ 0.03424603  0.96575298]
 ...
 [ 0.02960162  0.97039807]
 [ 0.01127756  0.98872113]
 [-0.01337702  1.01337665]]
Training Accuracy is :  92.320829907015  %
Testing has been finished!
The Total Testing Time is :  0.03

### 6. Accuracy and Fscore

In [862]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_y, predicted_list)
fscore = f1_score(test_y, predicted_list)

print(f"Accuracy: {accuracy:.2%}, Fscore: {fscore:.2%}")

Accuracy: 95.46%, Fscore: 78.13%
